# Domain knowledge steps in


After the talks with Dmitry I decided that every nan-value should be replaced with corresponding BAU value. Let's then do that and try same clusterings as before:

In [2]:
from gradutil import *
import pandas as pd
from pyomo.opt import SolverFactory

In [3]:
revenue, carbon, deadwood, ha = init_boreal()
x_revenue = nan_to_bau(revenue)
x_carbon = nan_to_bau(carbon)
x_deadwood = nan_to_bau(deadwood)
x_ha = nan_to_bau(ha)

In [4]:
x = np.concatenate((x_revenue.values, x_carbon.values, x_deadwood.values, x_ha.values), axis=1)
x_norm = normalize(x)

/home/jousi/Documents/gradujose/code/gradutil.py:40: RuntimeWarning: invalid value encountered in true_divide
  norm_data = np.where(normax != 0., norm_data / normax, 0)


In [5]:
opt = SolverFactory('glpk')
value_revenue, value_carbon, value_deadwood, value_ha = cNopt(x, x_norm, x, opt, nclust=10, seed=2)

In [6]:
value_revenue, value_carbon, value_deadwood, value_ha

(246209415.44215032,
 4430113.0166199952,
 214391.98103030928,
 19623.885137465888)

In [7]:
tmp_x = np.concatenate((x_ha.values,x_ha.values,x_ha.values,x_ha.values), axis=1)
tmp = cNopt(tmp_x, normalize(tmp_x), tmp_x, opt, nclust=100, seed=4)

In [8]:
centers, xtoc, dist = cluster(x_ha.values, 50, seed=3)

kmeans: X (29666, 7)  centres (50, 7)  delta=0.0001  maxiter=100  metric=cosine
kmeans: 100 iterations  cluster sizes: array([ 376,  677,  537,  220,  836,  397,  549,  252,  765,  876,   98,
       1451,  264,  722, 1278,  319,  345,  319,  624,  322,  292, 1089,
        875,  338,  822,  618,  562,  713,  393,  379, 1205,  442,  556,
        459,  383,  401, 1009,  727,  836,  605,  184,  479,  668,  748,
        455,  121, 1061,  103,  621, 1295])


In [9]:
%%time
weights = np.array([sum(xtoc==i) for i in range(len(centers))])
clustProblemHA = BorealWeightedProblem(centers,weights)
opt = SolverFactory('glpk')
resClustHA = opt.solve(clustProblemHA.model, False)

CPU times: user 5.1 s, sys: 8 ms, total: 5.11 s
Wall time: 5.12 s


In [10]:
val_ha = model_to_real_values(x_ha.values, xtoc, clustProblemHA.model)
real_ha = real_solutions()['ha']
(val_ha-real_ha)/real_ha

0.94938747060443573

So well, the Nan to BAU by itself didn't really help...